In [1]:
import requests
import pandas as pd
import json
import os 
from dotenv import load_dotenv
import joblib
import sqlite3
import time

In [2]:
load_dotenv()

GECKO_API = os.getenv("GECKO_API")
if not GECKO_API:
    raise ValueError("GECKO_API environment variable is not set.")

In [3]:
network = "eth"
token_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"  # USDT


# Format URL properly
url = f"https://pro-api.coingecko.com/api/v3/onchain/networks/{network}/tokens/{token_address}/pools"

# Get API key from environment variable
API_KEY = os.getenv("GECKO_API")
if not API_KEY:
    raise ValueError("GECKO_API environment variable not set.")

headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": API_KEY
}

# Make request
response = requests.get(url, headers=headers)
response.raise_for_status()
data = response.json()

In [4]:
# All USDT pair pools in the ethereum blockchain

all_pools = []

for pool in data["data"]:
    
    pool_data = {
        "id": pool["id"],
        "name": pool["attributes"]["name"],
        "pool_created_at": pool["attributes"]["pool_created_at"],
        "address": pool["attributes"]["address"]
    }
    all_pools.append(pool_data)

# Convert to DataFrame
df_pools = pd.DataFrame(all_pools)

df_pools.tail()

,id,name,pool_created_at,address
15,eth_0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,DAI / USDT 0.01%,2022-07-14T19:47:16Z,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406
16,eth_0x31373595f40ea48a7aab6cbcb0d377c6066e2dca,USDC / USDT 0.001%,2025-02-28T11:01:17Z,0x31373595f40ea48a7aab6cbcb0d377c6066e2dca
17,eth_0xd9f673912e1da331c9e56c5f0dbc7273c0eb6846...,USDC / USDT,2025-07-22T18:57:11Z,0xd9f673912e1da331c9e56c5f0dbc7273c0eb68461793...
18,eth_0x6ca298d2983ab03aa1da7679389d955a4efee15c,WETH / USDT 0.05%,2023-04-02T04:55:54Z,0x6ca298d2983ab03aa1da7679389d955a4efee15c
19,eth_0xc275a7390966e4bcbf331b837cd7316c4a3efa83,USDf / USDT 0.01%,2025-02-21T10:43:47Z,0xc275a7390966e4bcbf331b837cd7316c4a3efa83


In [5]:
# Converting pool created at to datetime
# For easier calculations, sorting and resampling or grouping by time

# Convert column to datetime (UTC)
df_pools['pool_created_at'] = pd.to_datetime(df_pools['pool_created_at'], utc=True)

# Creating a new column for pool age in days
df_pools['pool_age_days'] = (pd.Timestamp.now(tz='UTC') - df_pools['pool_created_at']).dt.days

df_pools.head()

,id,name,pool_created_at,address,pool_age_days
0,eth_0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d4...,USDC / USDT 0.001%,2025-04-01 02:10:47+00:00,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,148
1,eth_0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT 0.3%,2021-12-29 12:36:12+00:00,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,1336
2,eth_0x00836fe54625be242bcfa286207795405ca4fd10,sUSDS / USDT,2025-04-18 22:52:10+00:00,0x00836fe54625be242bcfa286207795405ca4fd10,130
3,eth_0x72331fcb696b0151904c03584b66dc8365bc63f8...,ETH / USDT 0.05%,2025-01-27 22:08:23+00:00,0x72331fcb696b0151904c03584b66dc8365bc63f8a144...,211
4,eth_0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,DAI / USDC / USDT,2022-07-13 11:48:12+00:00,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,1141


In [8]:
# Extract fee into its own column (keep the % sign)

df_pools["fee"] = df_pools["name"].str.extract(r"(\d+\.?\d*%)")

# Remove the fee part from the name
df_pools["name"] = df_pools["name"].str.replace(r"\s\d+(\.\d+)?%", "", regex=True)

df_pools.head()

,id,name,pool_created_at,address,pool_age_days,fee
0,eth_0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d4...,USDC / USDT,2025-04-01 02:10:47+00:00,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,148,0.001%
1,eth_0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT,2021-12-29 12:36:12+00:00,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,1336,0.3%
2,eth_0x00836fe54625be242bcfa286207795405ca4fd10,sUSDS / USDT,2025-04-18 22:52:10+00:00,0x00836fe54625be242bcfa286207795405ca4fd10,130,NaN
3,eth_0x72331fcb696b0151904c03584b66dc8365bc63f8...,ETH / USDT,2025-01-27 22:08:23+00:00,0x72331fcb696b0151904c03584b66dc8365bc63f8a144...,211,0.05%
4,eth_0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,DAI / USDC / USDT,2022-07-13 11:48:12+00:00,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,1141,NaN


In [9]:
#  The joblib directory i want to save extracted pools data to

path = r'/home/realist/projects/DexTracker/backend/Database'
os.makedirs(path, exist_ok=True)

In [10]:
# Save pools dataframe as .pkl
joblib.dump(df_pools, os.path.join(path, "USDT-pairs_pools.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_pools.pkl']

In [11]:
# Not calling for the quote token since its a stablecoin
# base_token_price_usd → For universal USD pricing
# base_token_price_quote_token → For the actual trading rate in the pool

# All USDT pair price in the ethereum blockchain

pair_price = []

for price in data["data"]:
    
    token_data = {
        "name": price["attributes"]["name"],
        "base_price_usd": price["attributes"]["base_token_price_usd"],
        "base_quote_price": price["attributes"]["base_token_price_quote_token"]
    }
    pair_price.append(token_data)

# Convert to DataFrame
df_prices = pd.DataFrame(pair_price)

df_prices.tail()

,name,base_price_usd,base_quote_price
15,DAI / USDT 0.01%,0.996879377562191,0.9998027744
16,USDC / USDT 0.001%,0.997950891548995,1.0007019104
17,USDC / USDT,0.999496910705587,0.9944893881
18,WETH / USDT 0.05%,4591.84497510248,4605.472589842
19,USDf / USDT 0.01%,0.997272671411764,1.0048321123


In [12]:
# Save pool price dataframe as .pkl
joblib.dump(df_prices, os.path.join(path, "USDT-pairs_prices.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_prices.pkl']

In [13]:
# TVL (Liquidity for USDT pairs)

pair_tvl = []

for liquidity in data["data"]:
    
    pair_liquidity = {
        "name": liquidity["attributes"]["name"],
        "pair_reserve_in_usd": liquidity["attributes"]["reserve_in_usd"]
    }
    pair_tvl.append(pair_liquidity)

# Convert to DataFrame
df_tvl = pd.DataFrame(pair_tvl)

df_tvl.tail()

,name,pair_reserve_in_usd
15,DAI / USDT 0.01%,7548092.4775
16,USDC / USDT 0.001%,2934846.0171
17,USDC / USDT,6413066.3927
18,WETH / USDT 0.05%,16611765.3198
19,USDf / USDT 0.01%,23480476.9696


In [15]:
# Save pools tvl dataframe as .pkl
joblib.dump(df_tvl, os.path.join(path, "USDT-pairs_tvl.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_tvl.pkl']

In [14]:
# Market metrics (FDV for USDT pairs)

pair_fdv = []

for diluted_value in data["data"]:
    
    pair_fully_diluted_value = {
        "name": diluted_value["attributes"]["name"],
        "fdv_usd": diluted_value["attributes"]["fdv_usd"]
    }
    pair_fdv.append(pair_fully_diluted_value)

# Convert to DataFrame
df_fdv = pd.DataFrame(pair_fdv)

df_fdv.tail()

,name,fdv_usd
15,DAI / USDT 0.01%,81545788134.983
16,USDC / USDT 0.001%,81560091034.6422
17,USDC / USDT,82196734272.4426
18,WETH / USDT 0.05%,81553286367.5838
19,USDf / USDT 0.01%,81557887328.5064


In [16]:
# Save pools fdv dataframe as .pkl
joblib.dump(df_fdv, os.path.join(path, "USDT-pairs_fdv.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_fdv.pkl']

In [17]:
# Price change

pair_price_change = []

for change in data["data"]:
    attr = change["attributes"]
    price_change = attr.get("price_change_percentage", {})  # This gets the dictionary

    # Create a dictionary combining pool name and price changes
    pool_price_change = {
        "name": attr["name"],
        "m5": float(price_change.get("m5", 0)),
        "m15": float(price_change.get("m15", 0)),
        "m30": float(price_change.get("m30", 0)),
        "h1": float(price_change.get("h1", 0)),
        "h6": float(price_change.get("h6", 0)),
        "h24": float(price_change.get("h24", 0))
    }
    pair_price_change.append(pool_price_change)

# Convert to DataFrame
df_price_chg_pct = pd.DataFrame(pair_price_change)

df_price_chg_pct.tail()

,name,m5,m15,m30,h1,h6,h24
15,DAI / USDT 0.01%,0.00,-0.01,-0.59,0.01,-0.59,-0.59
16,USDC / USDT 0.001%,0.00,0.00,0.00,0.05,-0.53,0.02
17,USDC / USDT,0.00,0.00,0.00,0.00,0.41,0.21
18,WETH / USDT 0.05%,-0.10,-0.14,-0.20,-0.20,-0.57,-0.30
19,USDf / USDT 0.01%,0.02,0.01,0.02,-0.58,-0.58,0.02


In [18]:
# Save pools percentage change in price dataframe as .pkl
joblib.dump(df_price_chg_pct, os.path.join(path, "USDT-pairs_price_chg_pct.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_price_chg_pct.pkl']

In [19]:
pair_transactions = []

for tnxs in data["data"]:
    attr = tnxs["attributes"]
    tx = attr.get("transactions", {})

    pair_tx = {
        "name": attr["name"],
        "buys_5m": tx.get("m5", {}).get("buys", 0),
        "sells_5m": tx.get("m5", {}).get("sells", 0),
        "buyers_5m": tx.get("m5", {}).get("buyers", 0),
        "sellers_5m": tx.get("m5", {}).get("sellers", 0),
        "buys_15m": tx.get("m15", {}).get("buys", 0),
        "sells_15m": tx.get("m15", {}).get("sells", 0),
        "buyers_15m": tx.get("m15", {}).get("buyers", 0),
        "sellers_15m": tx.get("m15", {}).get("sellers", 0),
        "buys_1h": tx.get("h1", {}).get("buys", 0),
        "sells_1h": tx.get("h1", {}).get("sells", 0),
        "buyers_1h": tx.get("h1", {}).get("buyers", 0),
        "sellers_1h": tx.get("h1", {}).get("sellers", 0),
        "buys_24h": tx.get("h24", {}).get("buys", 0),
        "sells_24h": tx.get("h24", {}).get("sells", 0),
        "buyers_24h": tx.get("h24", {}).get("buyers", 0),
        "sellers_24h": tx.get("h24", {}).get("sellers", 0),
    }

    pair_transactions.append(pair_tx)

# Convert to DataFrame
df_transactions = pd.DataFrame(pair_transactions)

df_transactions.tail()

,name,buys_5m,sells_5m,buyers_5m,sellers_5m,buys_15m,sells_15m,buyers_15m,sellers_15m,buys_1h,sells_1h,buyers_1h,sellers_1h,buys_24h,sells_24h,buyers_24h,sellers_24h
15,DAI / USDT 0.01%,1,1,1.0,1.0,5,2,5.0,2.0,13,4,11.0,4.0,217,165,134.0,85.0
16,USDC / USDT 0.001%,3,3,1.0,3.0,3,3,1.0,3.0,4,9,2.0,9.0,219,271,113.0,134.0
17,USDC / USDT,0,0,0.0,0.0,0,0,0.0,0.0,1,0,1.0,0.0,55,44,40.0,28.0
18,WETH / USDT 0.05%,1,1,1.0,1.0,3,4,3.0,3.0,15,20,13.0,13.0,852,762,385.0,235.0
19,USDf / USDT 0.01%,0,2,0.0,2.0,1,4,1.0,4.0,3,8,3.0,8.0,91,133,71.0,117.0


In [20]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_transactions, os.path.join(path, "USDT-pairs_transactions.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_transactions.pkl']

In [21]:
pair_volume = []

for volumes in data["data"]:
    attr = volumes["attributes"]
    vol = attr.get("volume_usd", {})

    pool_vol = {
        "name": attr["name"],
        "vol_5m": float(vol.get("m5", 0)),
        "vol_15m": float(vol.get("m15", 0)),
        "vol_30m": float(vol.get("m30", 0)),
        "vol_1h": float(vol.get("h1", 0)),
        "vol_6h": float(vol.get("h6", 0)),
        "vol_24h": float(vol.get("h24", 0))
    }

    pair_volume.append(pool_vol)

# Convert to DataFrame
df_volume = pd.DataFrame(pair_volume)

df_volume.tail()

,name,vol_5m,vol_15m,vol_30m,vol_1h,vol_6h,vol_24h
15,DAI / USDT 0.01%,5.783944e+03,8.494495e+03,9.873886e+03,7.581500e+04,2.477646e+06,1.911330e+07
16,USDC / USDT 0.001%,2.523843e+06,2.523843e+06,2.523843e+06,3.223063e+06,1.345830e+07,4.704600e+07
17,USDC / USDT,0.000000e+00,0.000000e+00,0.000000e+00,3.990760e+04,3.656932e+06,1.502924e+07
18,WETH / USDT 0.05%,5.377370e+03,1.803847e+04,3.307212e+04,7.657597e+04,7.333229e+05,5.460233e+06
19,USDf / USDT 0.01%,3.093690e+03,4.294437e+03,8.932254e+03,2.115151e+04,9.226198e+05,3.364990e+06


In [22]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_volume, os.path.join(path, "USDT-pairs_volume.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_volume.pkl']

In [26]:
pair_dex = []

for dex_pool in data["data"]:
    dex_info = dex_pool.get("relationships", {}).get("dex", {}).get("data", {})
    pool_dex = {
        "name": pool["attributes"]["name"],
        "dex": dex_info.get("id", "unknown")  # fallback if missing
    }
    pair_dex.append(pool_dex)

df_dex = pd.DataFrame(pair_dex)

df_dex["name"] = df_dex["name"].str.replace(r"\s\d+(\.\d+)?%", "", regex=True)


df_dex.tail()

,name,dex
15,USDf / USDT,uniswap_v3
16,USDf / USDT,maverick-v2-eth
17,USDf / USDT,bunni-ethereum
18,USDf / USDT,pancakeswap-v3-ethereum
19,USDf / USDT,uniswap_v3


In [27]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_dex, os.path.join(path, "USDT-pairs_dex.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_dex.pkl']